# Train modulated model

### Imports

In [1]:
import os
if 'notebooks' in os.getcwd(): os.chdir('../..')  # change to main directory
print('Working directory:', os.getcwd() )

Working directory: /scratch/snx3000/bp000429/adrian_sensorium


In [2]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Prepare data loader

In [3]:
# loading the SENSORIUM+ dataset
# filenames = ['../data/static27204-5-13-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

# another dataset
# filenames = ['notebooks/data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

# all datasets with behavior data
basepath = "notebooks/data/"
filenames = [os.path.join(basepath, file) for file in os.listdir(basepath) if ".zip" in file ]
filenames = [file for file in filenames if 'static26872-17-20' not in file]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': True,
                 # 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 128,
                 'scale':.25,
                 'preload_from_merged_data':True,
                 'include_trial_id':True,
                 'include_history':True,
                 'include_gain':True,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [4]:
tier = 'train'
dataset_name = '21067-10-18'

for batch in dataloaders[tier][dataset_name]:
    break
    
for i, field in enumerate(batch._fields):
    print(f"{field}, {batch[i].shape}")

images, torch.Size([128, 4, 36, 64])
responses, torch.Size([128, 8372])
behavior, torch.Size([128, 3])
pupil_center, torch.Size([128, 2])
trial_id, torch.Size([128, 1])
history, torch.Size([128, 8372, 5])
gain, torch.Size([128, 1])


### Instantiate modulated model

In [5]:
import importlib
from sensorium.models import models
importlib.reload(models)

<module 'sensorium.models.models' from '/scratch/snx3000/bp000429/adrian_sensorium/sensorium/models/models.py'>

In [6]:
model_fn = 'sensorium.models.modulated_stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': True,
  'with_history':True,
  'with_gain':True,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

In [7]:
model

ModulatedFiringRateEncoder(
  (core): Stacked2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(4, 64, kernel_size=(9, 9), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): AdaptiveELU()
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=64, bias=False)
          (out_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): AdaptiveELU()
      )
      (layer2): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_dept

### Configure trainer

In [8]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 200,
                 'verbose': True,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 'track_training':True,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

### Train model

In [ ]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

correlation 0.021659302
poisson_loss 28039642.0


Epoch 1: 100%|██████████| 216/216 [01:15<00:00,  2.88it/s]


correlation 0.032460805
poisson_loss 38090884.0


Epoch 2: 100%|██████████| 216/216 [01:14<00:00,  2.88it/s]


correlation 0.05982633
poisson_loss 35142812.0


...

Epoch 83: 100%|██████████| 216/216 [01:14<00:00,  2.89it/s]


correlation 0.17893349
poisson_loss 15427276.0


Epoch 84: 100%|██████████| 216/216 [01:14<00:00,  2.89it/s]


correlation 0.17963554
poisson_loss 15412110.0


Epoch 85:  82%|████████▏ | 177/216 [01:01<00:13,  2.92it/s]

### Save model

In [ ]:
torch.save(model.state_dict(), 'notebooks/my_models/withHistory_withState_allRecs_model_1.pth')

In [ ]:
# this model requires include_behavior=True
# model.load_state_dict(torch.load('notebooks/my_models/withHistory_21067-10-18_model_1.pth'));

### Predict responses

In [ ]:
from sensorium.utility import prediction

In [ ]:
# calculate predictions per dataloader
results = prediction.all_predictions_with_trial(model, dataloaders)

# merge predictions, sort in time and add behavioral variables
merged = prediction.merge_predictions(results)
sorted_res = prediction.sort_predictions_by_time(merged)
prediction.inplace_add_behavior_to_sorted_predictions(sorted_res)

In [ ]:
out_folder = 'notebooks/my_results'
# file_name = 'res_v1_allBehavior.npy'
file_name = 'res_v2_withModulatorAndState_all.npy'

np.save( os.path.join(out_folder, file_name), sorted_res)

In [ ]:
from ipylab import JupyterFrontEnd

app = JupyterFrontEnd()
app.commands.execute('docmanager:save')